In [3]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
import panel as pn

pn.extension()

%matplotlib inline


In [4]:
# Local imports
import sys
sys.path.append("../libs")

from Constants import Constants
from ProcessingTools import DateProcessingTools


In [5]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)


In [6]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)


In [7]:
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

In [8]:
atlas.head()


,Number of Users,Invoice Date,Invoice #,Year,Month,Invoice Amount,Subscription,2021 Prediction,Customers Status,Account Code,Address,State,Lat,Long,Service Start,Service End,Subscription Duration
Customers,,,,,,,,,,,,,,,,,
University 1,7050.0,2015-03-20 00:00:00-04:00,ATLAS 315,2015,March,72000.0,1 Year,NaN,Lost,4700-0-00-00000-18-0000,"1201 N 3rd St #6, Baton Rouge, LA 70802",LA,30.460190,-91.188530,2015-03-18 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 2,250.0,2015-05-28 00:00:00-04:00,AJ501,2015,May,3500.0,1 Year,NaN,Active,4700-0-00-00000-16-0000,"1 Kellogg Cir, Emporia, KS 66801",KS,38.392609,-96.181396,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 3,100.0,2015-06-23 00:00:00-04:00,AJ502,2015,June,3500.0,1 Year,NaN,Active,4700-0-00-00000-17-0000,"150 University Blvd, Morehead, KY 40351",KY,38.184921,-83.434441,2015-06-10 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 4,500.0,2015-06-26 00:00:00-04:00,AJ503,2015,June,6500.0,1 Year,NaN,Lost,4700-0-00-00000-32-0000,"44 Pierrepont Ave, Potsdam, NY 13676",NY,44.663780,-74.978409,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 5,15.0,2015-10-07 00:00:00-04:00,AJ504,2015,October,750.0,1 Year,NaN,Lost,4700-0-00-00000-20-0000,"47645 College Dr, St Marys City, MD 20686",MD,38.190601,-76.426300,2015-10-05 00:00:00-04:00,2016-09-30 00:00:00-04:00,365 days


In [9]:
total_data = atlas.loc[:, ["Number of Users", "Invoice Amount"]].groupby(["Customers"]).sum()
# total_data.dropna(inplace=True)
total_data

atlas_with_totals = pd.concat([atlas.drop(columns=["Number of Users", "Invoice Amount"]), total_data], axis="columns", join="inner")

In [10]:
px.set_mapbox_access_token(mapbox_token)
map = px.scatter_mapbox(
    atlas_with_totals.reset_index(),
    lat="Lat",
    lon="Long",
    # size="Invoice Amount",
    size="Invoice Amount",
    color="Customers",
    width=1800,
    height=900,
    zoom=4,
    mapbox_style="open-street-map"  # Options: [ "open-street-map, white-bg, carto-positron, carto-darkmatter, stamen-terrain, stamen-toner, stamen-watercolor" ]
)
map.show()


In [11]:
# Group by year and neighborhood and then create a new dataframe of the mean values
atlas_with_totals["ServiceStart_Year"] = atlas_with_totals["Service Start"].apply(lambda x: x.year)
atlas_with_totals_grouped = atlas_with_totals.groupby(["ServiceStart_Year", "State"]).sum()
atlas_with_totals_grouped.head(5)

def generate_map(state):
    data = atlas_with_totals.reset_index().set_index("State").loc[state]
    map = px.scatter_mapbox(
        data.reset_index(),
        lat="Lat",
        lon="Long",
        size="Invoice Amount",
        # size="Number of Users",
        color="Customers",
        width=1800,
        height=900,
        zoom=4,
        mapbox_style="open-street-map"  # Options: [ "open-street-map, white-bg, carto-positron, carto-darkmatter, stamen-terrain, stamen-toner, stamen-watercolor" ]
    )
    return map.show()

interact(generate_map, state=constants.DROP_DOWNS_STATE_ABBREVIATION_LIST)

Column
    [0] Column
        [0] Select(name='state', options=['AL', 'AK', 'AZ', ...], value='AL')
    [1] Row
        [0] Str(None, name='interactive01450')